# Labeled Property Graph Play

The notebook looks at working with the LPG from SysML v2 clients

In [1]:
import pymbe.api as pm
from pymbe.label import get_label
import networkx as NX
import matplotlib.pyplot as plt

from pymbe.interpretation.interp_playbooks import random_generator_playbook
from pymbe.interpretation.results import pprint_interpretation

from pymbe.query.metamodel_navigator import map_inputs_to_results

In [2]:
pm.UI.new(host_url="http://sysml2-sst.intercax.com")

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:267: UserWarning: These edge types are not in the graph: {'FeatureMembership', 'ParameterMembership', 'ResultExpressionMembership', 'ReturnParameterMembership', 'FeatureValue'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


DockPop(children=(UI(children=(SysML2ClientWidget(children=(Text(value='http://sysml2-sst.intercax.com', descr…

In [3]:
ui, *_ = _.children
client, tree, inspector, lpg, interpreter = ui.children
client.project_selector.value = client.project_selector.options["Kerbal (2021-04-17 18:03:01)"]
client._download_elements()

In [4]:
client.project_selector.value

'db6f06e9-71af-4ec3-98f2-03548516ed5d'

In [5]:
client.commit_selector.value

'69116641-5991-4974-9d85-69d8a9e8eee5'

In [6]:
from copy import deepcopy
from uuid import uuid4
from pymbe.query.metamodel_navigator import map_inputs_to_results


new_edges = [
    (source, target, metatype, {
        "@id": f"_{uuid4()}",
        "@type": metatype,
        "label": metatype,
        "relatedElement": [
            {"@id": source},
            {"@id": target},
        ],
        "source": [{"@id": source}],
        "target": [{"@id": target}],
    })
    for source, target, metatype in map_inputs_to_results(lpg)
]
new_elements = {
    data["@id"]: data
    for *_, data in new_edges
}
new_edges

client.elements_by_id = {**client.elements_by_id, **new_elements}

In [7]:
shorten_pre_bake = {
    'RT-10 "Hammer" Solid Fuel Booster': "RT-10",
    'RT-5 "Flea" Solid Fuel Booster': "RT-5",
    'LV-T45 "Swivel" Liquid Fuel Engine': "LV-T45",
    'FL-T100 Fuel Tank': "FL-T100",
    'FL-T200 Fuel Tank': "FL-T200"
}

In [8]:
random_rez = random_generator_playbook(
    lpg,
    shorten_pre_bake
)

In [9]:
from pymbe.interpretation.interp_playbooks import build_sequence_templates
from pymbe.interpretation.results import pprint_single_id_list
from pymbe.interpretation.results import pprint_double_id_list
pprint_double_id_list(build_sequence_templates(lpg), lpg.nodes)

[['Rocket', 'stages: Rocket Stage', 'Coupler to Carrying Stage: Coupler'],
 ['Solid Stage', 'boosters: Solid Booster'],
 ['Liquid Stage', 'engines: Liquid Engine'],
 ['Liquid Stage', 'tanks: Fuel Tank Section']]

In [10]:
part_featuring_graph = lpg.get_projection("Part Featuring Graph")

In [11]:
part_featuring_graph

In [12]:
feature_components = NX.connected_components(part_featuring_graph.to_undirected())

In [13]:
connected_subs = []
for comp in NX.connected_components(part_featuring_graph.to_undirected()):
    connected_sub = NX.subgraph(part_featuring_graph, list(comp))
    connected_subs.append(connected_sub)

In [14]:
for connected in connected_subs:
    #print(pprint_single_id_list(list(connected.nodes), lpg.nodes))
    leaves = [node for node in connected.nodes if connected.in_degree(node) == 0]
    roots = [node for node in connected.nodes if connected.out_degree(node) == 0]
    print(pprint_single_id_list(leaves, lpg.nodes))

['Rocket']
['Solid Stage']
['Liquid Stage']


In [15]:
from pymbe.local.bundle import download_to_local

download_to_local(
    "http://sysml2-sst.intercax.com",
    "Kerbal",
    client.project_selector.value,
    client.commit_selector.value,
    "C:/Users/bjorn/Documents/Project Dany"
)

http://sysml2-sst.intercax.com:9000/projects/db6f06e9-71af-4ec3-98f2-03548516ed5d/commits/69116641-5991-4974-9d85-69d8a9e8eee5/elements?page[size]=5000
